# Finding possible game states in tic-tac-toe and their symmetry groups

Code in this notebook is copied from https://github.com/vug/tic-tac-toe-game-states-graph unless otherwise specified.

## Imports and data structures

In [1]:
from collections import deque
from typing import Iterable, List

#
# Data Structures
#

# Represents one board configuration.
# Other constraints of a State type
#   inner tuple is made of 3 integers, a triplet made of {0, 1, 2}
#   and outer tuple is made of 3 triplets.
State = Iterable[Iterable[int]]
# Constraint on SymmetryGroup type is that
# states it includes has to be symmetric under
# rotation and mirroring.
# Also first state in the list is the
# canonical representation of the group.
SymmetryGroup = List[State]
# Represent the ID of individual Symmetry Groups
GroupId = int


class Node(object):
    def __init__(self, val: GroupId = None, successors: List[GroupId] = None):
        self.val = val
        self.successors = successors or []


## Debug utilities

In [2]:

# I'd prefer a chain of operations piped sequentially.
# But, here, the processing starts from the inner loop
# I can't join cells in a row, before they are converted to strings

def state_str(st: State) -> str:
    def row_to_str(row: Iterable[int]):
        row = [str(num) for num in row]
        return "".join(row)

    return "\n".join(row_to_str(row) for row in st)


def print_state(st: State) -> None:
    print(state_str(st))


def print_states(states: Iterable[State]):
    for st in states:
        print_state(st)
        print()



## Algorithms

In [3]:

def get_symmetries(st: State) -> SymmetryGroup:
    """Compute Symmetry Group of a board state.

    There are 8 symmetries of a state. Identity + 3 90 degree
    rotations. And the same for it's mirror (either horizontal
    or vertical.)

    Also remove duplicate states in a group.

    Keeps the original state to be used to compute the group
    at the first index, so that it can be used as the
    "canonical state" that represents the group.
    """

    def get_vertical_mirror(st: State) -> State:
        return tuple(reversed(st))

    def get_rotations(st: State) -> List[State]:
        # fmt: off
        ((a, b, c),
         (d, e, f),
         (g, h, i)) = st
        rot1 = ((g, d, a),
                (h, e, b),
                (i, f, c))
        rot2 = ((i, h, g),
                (f, e, d),
                (c, b, a))
        rot3 = ((c, f, i),
                (b, e, h),
                (a, d, g))
        # fmt: on
        return [st, rot1, rot2, rot3]

    def apply_all_symmetry_operations(st: State) -> List[State]:
        mir = get_vertical_mirror(st)
        return get_rotations(st) + get_rotations(mir)

    all_symmetries = apply_all_symmetry_operations(st)
    duplicates_removed = list(dict.fromkeys(all_symmetries))
    return duplicates_removed



In [4]:

def is_end(st: State) -> bool:
    """Tell whether given state is a game-ending state.

    Go over every column, row and diagonals (lines) and
    check whether they are made of all-Xs or all-Os.
    """

    def are_same(triplet):
        a, b, c = triplet
        return a == b == c != 0

    horizontals = [[st[ix][0], st[ix][1], st[ix][2]] for ix in range(3)]
    verticals = [[st[0][ix], st[1][ix], st[2][ix]] for ix in range(3)]
    diag1 = [st[0][0], st[1][1], st[2][2]]
    diag2 = [st[2][0], st[1][1], st[0][2]]
    lines = horizontals + verticals + [diag1, diag2]

    return any(are_same(line) for line in lines)



In [5]:

def make_move(st: State, row: int, col: int, val: int) -> State:
    """Take a state and a move and creates resulting state.

    Do not check the legality of the move."""
    mutable_st = [list(r) for r in st]
    mutable_st[row][col] = val
    new_st = tuple(tuple(r) for r in mutable_st)
    return new_st



In [6]:

def get_next_states_raw(st: State, step: int) -> List[State]:
    """Compute states that can be reach via legal moves.

    Given a state and the number of current step.
    Assuming empty board is step=0, X's are placed in even
    steps and O's in odd steps.

    If given state is an ending state, there no next states.
    """
    next_states = []
    if is_end(st):
        return []
    new_val = step % 2 + 1
    for row in range(3):
        for col in range(3):
            val = st[row][col]
            if val == 0:
                new_st = make_move(st, row, col, new_val)
                next_states.append(new_st)
    return next_states

## Calculating possible board states

Here we set up the initial board state and add first symmetry group.

In [7]:
empty_board_state = ((0, 0, 0), (0, 0, 0), (0, 0, 0))
# index from states (hashable since tuple) to symmetry group ids
states = {}
# index from symmetry group id to actual groups
groups = {}
# index from symmetry group id to Node that hold them in the DAG
nodes = {}
# Incremental counter of GroupIds
gid = 1

def add_symmetry_group(st: State, gid: GroupId):
    """Compute symmetry group, update global state.

    Should be called when a new unique state is encountered.
    Register computed group into `groups` via given `gid`.
    Store the belonging of the states of the group in `states.
    """
    global states, groups
    assert gid not in groups
    symmetry_group = get_symmetries(st)
    groups[gid] = symmetry_group
    states.update({st: gid for st in symmetry_group})

# Handle initial state
add_symmetry_group(empty_board_state, gid)
root = Node(gid)
nodes[gid] = root
gid += 1
q = deque([root])

In [8]:
print_states(states) #Print out of initial board state

000
000
000



We then define funtion that finds possible following game states and identifies symmetry groups from these...

In [9]:
def compute_and_queue_successors(nd: Node) -> None:
    global groups, nodes, q, gid
    sym_group = groups[nd.val]
    # first state of a group is its representative state
    canonical_state = sym_group[0]
    next_states = get_next_states_raw(canonical_state, step)
    for nxt_st in next_states:
        # have seen/processed this state before?
        if nxt_st in states:
            gid_old = states[nxt_st]
            # merge this path with other paths leading to this node
            # if it is not already merged (i.e. if not a successor
            # of current node)
            if gid_old not in [suc.val for suc in nd.successors]:
                nd.successors.append(nodes[gid_old])
            continue
        # is it an unseen/unprocessed state?
        add_symmetry_group(nxt_st, gid)
        new_nd = Node(gid)
        nodes[gid] = new_nd
        nd.successors.append(new_nd)
        q.appendleft(new_nd)
        gid += 1

... and calculates symmetry groups for all ten possible turns of the game.

In [10]:
print("step, unique_state_no")
for step in range(11):
    print(f"{step}, {len(q)}")
    for _ in range(len(q)):
        compute_and_queue_successors(q.pop())

step, unique_state_no
0, 1
1, 3
2, 12
3, 38
4, 108
5, 174
6, 204
7, 153
8, 57
9, 15
10, 0


--------------------------
Code from here on down is written by me

There are 765 possible symmetry groups (unique states considering symmetry) in total.

In [11]:
len(groups)

765

groups is a dict of lists containing all states within each symmetry group.

In [12]:
groups[2]

[((1, 0, 0), (0, 0, 0), (0, 0, 0)),
 ((0, 0, 1), (0, 0, 0), (0, 0, 0)),
 ((0, 0, 0), (0, 0, 0), (0, 0, 1)),
 ((0, 0, 0), (0, 0, 0), (1, 0, 0))]

This is the same as above but printed in a more readable way:

In [13]:
print_states(groups[2])

100
000
000

001
000
000

000
000
001

000
000
100



There is a total of 5478 possible states in the game (not accounting for symmetry).

In [14]:
s=0
for g in range(1, len(groups)+1):
  s += len(groups[g])
s, len(states)

(5478, 5478)

## Using board states in MENACE

Our MENACE will be trained for the 765 symmetry groups. The original match box MENACE used 304 boxes as it always played first and only considered states relevant for player 1 i.e. even steps in the uniqe state count above. The slightly lower match box count compared to possible states for player 1 is probably explained by exclusion of end game states (I guess).

The game engine will compare the current game state to the 5478 possible states within ```groups``` above and identify the symmetry group from the 765 possibilities.  
MENACE will then play according to its own algorithm for that particular symmetry group (match box).





Now we write a function that calculates all possible board states as above and returns a dictionary with all states and their corresponding symmetry group index. This function can be imported to other notebooks.

In [15]:
'''
This code is not excellent given all the global variables. But it works.
'''

def all_states_and_groups():
  # Reset global variables
  global states, groups, nodes, gid, q

  empty_board_state = ((0, 0, 0), (0, 0, 0), (0, 0, 0))
  # index from states (hashable since tuple) to symmetry group ids
  states = {}
  # index from symmetry group id to actual groups
  groups = {}
  # index from symmetry group id to Node that hold them in the DAG
  nodes = {}
  # Incremental counter of GroupIds
  gid = 1


  # Handle initial state
  add_symmetry_group(empty_board_state, gid)
  root = Node(gid)
  nodes[gid] = root
  gid += 1
  q = deque([root])

  # Calculate possible board states
  global step
  for step in range(11):
    for _ in range(len(q)):
        compute_and_queue_successors(q.pop())
  return states

In [16]:
all_states_and_groups()

{((0, 0, 0), (0, 0, 0), (0, 0, 0)): 1,
 ((1, 0, 0), (0, 0, 0), (0, 0, 0)): 2,
 ((0, 0, 1), (0, 0, 0), (0, 0, 0)): 2,
 ((0, 0, 0), (0, 0, 0), (0, 0, 1)): 2,
 ((0, 0, 0), (0, 0, 0), (1, 0, 0)): 2,
 ((0, 1, 0), (0, 0, 0), (0, 0, 0)): 3,
 ((0, 0, 0), (0, 0, 1), (0, 0, 0)): 3,
 ((0, 0, 0), (0, 0, 0), (0, 1, 0)): 3,
 ((0, 0, 0), (1, 0, 0), (0, 0, 0)): 3,
 ((0, 0, 0), (0, 1, 0), (0, 0, 0)): 4,
 ((1, 2, 0), (0, 0, 0), (0, 0, 0)): 5,
 ((0, 0, 1), (0, 0, 2), (0, 0, 0)): 5,
 ((0, 0, 0), (0, 0, 0), (0, 2, 1)): 5,
 ((0, 0, 0), (2, 0, 0), (1, 0, 0)): 5,
 ((0, 0, 0), (0, 0, 0), (1, 2, 0)): 5,
 ((1, 0, 0), (2, 0, 0), (0, 0, 0)): 5,
 ((0, 2, 1), (0, 0, 0), (0, 0, 0)): 5,
 ((0, 0, 0), (0, 0, 2), (0, 0, 1)): 5,
 ((1, 0, 2), (0, 0, 0), (0, 0, 0)): 6,
 ((0, 0, 1), (0, 0, 0), (0, 0, 2)): 6,
 ((0, 0, 0), (0, 0, 0), (2, 0, 1)): 6,
 ((2, 0, 0), (0, 0, 0), (1, 0, 0)): 6,
 ((0, 0, 0), (0, 0, 0), (1, 0, 2)): 6,
 ((1, 0, 0), (0, 0, 0), (2, 0, 0)): 6,
 ((2, 0, 1), (0, 0, 0), (0, 0, 0)): 6,
 ((0, 0, 2), (0, 0, 0), (

We create a copy of this file with only function definitions to work as import to other notebook.